In [1]:
import numpy as np
import open3d as o3d
import correct_lidar

import glob

import altair as alt
alt.data_transformers.enable('json')
import seaborn as sns
import matplotlib.pyplot as plt 
import pandas as pd
import datetime as dt

import xarray as xr
from joblib import Parallel, delayed
from tqdm import tqdm
from sublimpy import utils

In [2]:
fp = "/Users/elischwat/Development/data/sublimationofsnow/lidar_raw/l1/2022-12-31/2022-12-31_19-05-16.lvx"
output_prefix = fp.split('/')[-1].replace('.lvx', '')

In [3]:
# Open SOS Measurement Dataset
################################################
start_date = '20221130'
end_date = '20230509'
# open files
tidy_df = pd.read_parquet(f'../paper1/tidy_df_{start_date}_{end_date}_noplanar_fit.parquet')

# convert time column to datetime
tidy_df['time'] = pd.to_datetime(tidy_df['time'])
tidy_df = utils.modify_df_timezone(tidy_df, 'UTC', 'US/Mountain')

# Process single file

## Open a single scan and save to file

In [4]:
points = correct_lidar.read_file(fp)
# points = points.T
# combined_points_bs = np.concatenate(loaded)

In [5]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[:,:3])

In [6]:
o3d.io.write_point_cloud(f"{output_prefix}.ply", pcd, format = 'ply')

True

## Rotate points to be in earth-relative coordinate system, save

In [7]:
theta = np.deg2rad(25)
points_rotated = []

for i, [x, y, z] in enumerate(points[:,:3]):
    x_ = x*np.cos(theta) + y*np.sin(theta)
    y_ = -x*np.sin(theta) + y*np.cos(theta)
    points_rotated.append(np.array([x_, y_, z]))
points_rotated = np.array(points_rotated)

In [8]:
pcd_rot = o3d.geometry.PointCloud()
pcd_rot.points = o3d.utility.Vector3dVector(points_rotated)
o3d.io.write_point_cloud(f"{output_prefix}_rot.ply", pcd_rot, format = 'ply')

True

## Filter points and save

In [9]:
points_filtered = np.array([
    np.array([x, y, z]) for [x,y,z] in points_rotated
    if x <= 14 and z < 0.6 and z > -3.69
])

In [10]:
pcd_rot_filt = o3d.geometry.PointCloud()
pcd_rot_filt.points = o3d.utility.Vector3dVector(points_filtered)
o3d.io.write_point_cloud(f"{output_prefix}_rot_filt.ply", pcd_rot_filt, format = 'ply', )

True

In [11]:
print("min/max x values:", round(points_filtered[:, 0].min(), 3), round(points_filtered[:, 0].max(), 3))
print("min/max y values:", round(points_filtered[:, 1].min(), 3), round(points_filtered[:, 1].max(), 3))
print("min/max z values:", round(points_filtered[:, 2].min(), 3), round(points_filtered[:, 2].max(), 3))

min/max x values: 0.031 13.999
min/max y values: -9.36 1.27
min/max z values: -3.689 0.599


## Remove "ground" points, using a 5cm buffer, and save

In [12]:
points_ground_filter = np.array([
    np.array([x, y, z]) for [x,y,z] in points_filtered
    if y > -8.8
])
points_ground_filter

array([[ 3.21296873, -1.04364022,  0.274     ],
       [ 3.29977392, -1.22424714,  0.195     ],
       [ 4.3539613 , -1.81954032,  0.135     ],
       ...,
       [ 2.56941674, -1.31399338,  0.262     ],
       [ 2.53532633, -1.45808656,  0.567     ],
       [ 2.53091693, -1.46044394,  0.575     ]])

In [13]:
pcd_rot_filt_ground = o3d.geometry.PointCloud()
pcd_rot_filt_ground.points = o3d.utility.Vector3dVector(points_ground_filter)
o3d.io.write_point_cloud(f"{output_prefix}_rot_filt_ground.ply", pcd_rot_filt_ground, format = 'ply', )

True

## Unrotate the filtered points

In [14]:
points_ground_filter_unrotated = []

for i, [x, y, z] in enumerate(points_ground_filter):
    x_ = x*np.cos( - theta) + y*np.sin( - theta)
    y_ = -x*np.sin( - theta) + y*np.cos( - theta)
    points_ground_filter_unrotated.append(np.array([x_, y_, z]))
points_ground_filter_unrotated = np.array(points_ground_filter_unrotated)

In [15]:
pts_df = pd.DataFrame(points_ground_filter_unrotated).rename(columns = {
    0: 'x',
    1: 'y',
    2: 'z',
})

# Calculate distance from centerline (e.g. radius)
pts_df['r'] = np.sqrt(pts_df['y']**2 + pts_df['z']**2)

In [16]:
pts_df.head(3)

,x,y,z,r
0,3.353,0.412,0.274,0.494793
1,3.508,0.285,0.195,0.345326
2,4.715,0.191,0.135,0.233893


In [17]:
alt.Chart(pts_df).mark_bar().encode(
    alt.X("x:Q").bin(),
    alt.Y("count():Q")
).properties(width = 200, height = 200) |\
alt.Chart(pts_df).mark_bar().encode(
    alt.X("r:Q").bin(),
    alt.Y("count():Q")
).properties(width = 200, height = 200)

alt.HConcatChart(...)

In [18]:
alt.Chart(pts_df).transform_density(
    'x',
    as_=['x', 'density'],
).mark_line().encode(
    x="x:Q",
    y='density:Q',
).properties(width = 200, height = 200)

alt.Chart(...)

# Process multiple files

In [19]:
theta = np.deg2rad(25)

define function to isolate points that are actually in the air

In [20]:
def get_filtered_points(fp):
    points = correct_lidar.read_file(fp)[:,:3]

    # Rotate the points so the ground is ~horizontal
    points_rotated = []
    for i, [x, y, z] in enumerate(points):
        x_ = x*np.cos(theta) + y*np.sin(theta)
        y_ = -x*np.sin(theta) + y*np.cos(theta)
        points_rotated.append(np.array([x_, y_, z]))
    points_rotated = np.array(points_rotated)

    # Filter the points to remove physical obstructions
    points_filtered = np.array([
        np.array([x, y, z]) for [x,y,z] in points_rotated
        if x <= 14 and z < 0.6 and z > -3.69
    ])

    # Filter the points to remove the ground
    points_ground_filter = np.array([
        np.array([x, y, z]) for [x,y,z] in points_filtered
        if y > -8.8
        # -9.331 - for 12/12/22 case study
        # -8.8 - for 12/31/22 case study
    ])

    # Unrotate so we are back in the original coordinate system.
    points_ground_filter_unrotated = []
    for i, [x, y, z] in enumerate(points_ground_filter):
        x_ = x*np.cos( - theta) + y*np.sin( - theta)
        y_ = -x*np.sin( - theta) + y*np.cos( - theta)
        points_ground_filter_unrotated.append(np.array([x_, y_, z]))
    points_ground_filter_unrotated = np.array(points_ground_filter_unrotated)

    pts_df = pd.DataFrame(points_ground_filter_unrotated).rename(columns = {
        0: 'x',
        1: 'y',
        2: 'z',
    })
    pts_df['file'] = fp
    pts_df['date_str'] = fp.split('/')[-1].replace('.lvx', '')
    return pts_df

for two case studies, open all lidar files and filter the points we want

In [85]:
PARALLELISM = 8

###### BLOWING SNOW CASE STUDY ######
# Dec. 12 1200 - Dec. 13 2300
FILE_LIST_1 = glob.glob("/Users/elischwat/Development/data/sublimationofsnow/lidar_raw/l1/2022-12-12/*.lvx") +\
    glob.glob("/Users/elischwat/Development/data/sublimationofsnow/lidar_raw/l1/2022-12-13/*.lvx") + \
    glob.glob("/Users/elischwat/Development/data/sublimationofsnow/lidar_raw/l1/2022-12-14/*.lvx")
FILE_LIST_1 = sorted([f for f in FILE_LIST_1 if "05-17.lvx" in f or "05-16.lvx" in f or "05-15.lvx" in f])
FILE_LIST_1 = FILE_LIST_1[0+12+7:36+12+7]

###### SNOWFALL CASE STUDY ######
# Dec. 31 0000-2400
FILE_LIST_2 = glob.glob("/Users/elischwat/Development/data/sublimationofsnow/lidar_raw/l1/2022-12-31/*.lvx") +\
glob.glob("/Users/elischwat/Development/data/sublimationofsnow/lidar_raw/l1/2023-01-01/*.lvx")
FILE_LIST_2 = sorted([f for f in FILE_LIST_2 if "05-17.lvx" in f or "05-16.lvx" in f or "05-15.lvx" in f])
FILE_LIST_2 = FILE_LIST_2[0+7:24+7]

ALL_FILES = FILE_LIST_1 + FILE_LIST_2
df_list =  Parallel(n_jobs = PARALLELISM)(
        delayed(get_filtered_points)(fp) 
        for fp in tqdm(ALL_FILES)
    )
pts_df = pd.concat(df_list)
pts_df['time'] = pd.to_datetime(pts_df['date_str'], format='%Y-%m-%d_%H-%M-%S')
pts_df['time utc'] = pts_df['time']
pts_df = utils.modify_df_timezone(pts_df, 'UTC', 'US/Mountain')
pts_df.time = pts_df.time.apply(lambda dt: dt.replace(second=0))
pts_df['date_str_local'] = pts_df['time'].dt.strftime('%Y-%m-%d_%H-%M-%S')

pts_df = pts_df.reset_index(drop=True)

  0%|          | 0/60 [00:00<?, ?it/s]

100%|██████████| 60/60 [00:26<00:00,  2.25it/s]


For two case studies, open up kettle ponds data

In [86]:
sos_base_path = "/Users/elischwat/Development/data/sublimationofsnow/sosqc/sos_qc_geo_tiltcor_v20240307/"
sos_df_1 = xr.concat([
    xr.open_dataset(os.path.join(sos_base_path, "isfs_sos_qc_geo_tiltcor_5min_20221212.nc")),
    xr.open_dataset(os.path.join(sos_base_path, "isfs_sos_qc_geo_tiltcor_5min_20221213.nc")),
    xr.open_dataset(os.path.join(sos_base_path, "isfs_sos_qc_geo_tiltcor_5min_20221214.nc")),
], dim='time')[['SF_avg_1m_ue', 'SF_avg_2m_ue']].to_dataframe().reset_index()
sos_df_1 = utils.modify_df_timezone(sos_df_1, 'UTC', 'US/Mountain')
sos_df_1.time = sos_df_1.time - dt.timedelta(minutes = 2, seconds = 30)
# isolate dates to Dec. 12 1200 – Dec 13 2300
sos_df_1 = sos_df_1[
    (sos_df_1.time >= "2022-12-12 1200") &
    (sos_df_1.time <= "2022-12-13 2355")
]

sos_df_2 = xr.concat([
    xr.open_dataset(os.path.join(sos_base_path, "isfs_sos_qc_geo_tiltcor_5min_20221231.nc")),
    xr.open_dataset(os.path.join(sos_base_path, "isfs_sos_qc_geo_tiltcor_5min_20230101.nc")),
], dim='time')[['SF_avg_1m_ue', 'SF_avg_2m_ue']].to_dataframe().reset_index()
sos_df_2 = utils.modify_df_timezone(sos_df_2, 'UTC', 'US/Mountain')
sos_df_2.time = sos_df_2.time - dt.timedelta(minutes = 2, seconds = 30)
# isolate dates to Dec. 31 0000-2400
sos_df_2 = sos_df_2[
    (sos_df_2.time >= "2022-12-31 0000") &
    (sos_df_2.time <= "2022-12-31 2355")
]

# combine datasets
sos_df = pd.concat([sos_df_1, sos_df_2])

In [87]:
sos_df['is_blowing_snow'] = (sos_df['SF_avg_1m_ue'] > 0) | (sos_df['SF_avg_2m_ue'] > 0)
sos_df['is_high_blowing_snow'] = (sos_df['SF_avg_1m_ue'] > 0.25) | (sos_df['SF_avg_2m_ue'] > 0.25)

Calculate KDE for each timestamp

In [88]:
from scipy.stats import gaussian_kde

In [89]:
xs = np.linspace(0,20)
pts_df_filtered = pts_df.groupby(['time', 'date_str_local']).filter(lambda df: len(df) > 10)
kde_df = pd.DataFrame(
    pts_df_filtered.groupby(['time', 'date_str_local'])['x'].apply(lambda x_series: gaussian_kde(x_series)(xs))
).rename(columns={'x': 'kde'})
kde_df['x'] = kde_df.apply(lambda x: xs, axis=1)
kde_df = kde_df.explode(column=['kde', 'x']).reset_index()
kde_df = kde_df.merge(
    sos_df[['time', 'is_blowing_snow', 'is_high_blowing_snow']],
    on='time',
    how='left'
)

In [91]:
kde_df['case_study'] = kde_df.time < "2022-12-15"
kde_df['case_study'] = kde_df['case_study'].apply(lambda early: 1 if early else)

In [104]:
(    
    alt.Chart(
        kde_df[kde_df.time < "2022-12-15"]    
    ).mark_line().encode(
        x=alt.X("x:Q").title("distance from lidar along central axis (m)"),
        y=alt.Y('kde:Q').title("KDE of point count"),
        color=alt.Color('date_str_local:O').scale(scheme='turbo'),
        facet=alt.Facet("is_blowing_snow:N").title("FlowCapt blowing snow flux > 0.0"),
        tooltip = 'date_str_local'
    ).properties(width = 200, height = 200, title='Blowing snow case study, Dec 12 1200 - Dec 13 2300')
    &
    alt.Chart(
        kde_df[kde_df.time > "2022-12-15"]    
    ).mark_line().encode(
        x=alt.X("x:Q").title("distance from lidar along central axis (m)"),
        y=alt.Y('kde:Q').title("KDE of point count"),
        color=alt.Color('date_str_local:O').scale(scheme='turbo'),
        facet=alt.Facet("is_blowing_snow:N").title("FlowCapt blowing snow flux > 0.0"),
        tooltip = 'date_str_local'
    ).properties(width = 200, height = 200, title='Snowfall case study, Dec 31 0000 - 2300')
).resolve_scale(color='independent').configure_legend(columns=2, symbolLimit=100).display(renderer='svg')

alt.VConcatChart(...)